In [1]:
import json
import random
from datasets import load_dataset

import warnings
warnings.filterwarnings("ignore")

DATASET = "allenai/c4"
SUBSET = "en"               # default subset
SPLIT = "train"
STREAMING = True

OUTPUT_PATH = "pretrain_c4.jsonl"

MAX_SAMPLES = 500_000       # 5L
MAX_CHARS = 512             # strict limit

CHUNK_SIZE = 40_000         # ensures strong global shuffling


def write_chunk(chunk, path):
    """Shuffle chunk & append to disk."""
    random.shuffle(chunk)
    with open(path, "a", encoding="utf-8") as f:
        for item in chunk:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")


def main():
    print(f"🔄 Streaming dataset: {DATASET} ({SUBSET} subset)")

    ds = load_dataset(
        DATASET,
        name=SUBSET,
        split=SPLIT,
        streaming=STREAMING
    )

    buffer = []
    written = 0
    total_chars = 0
    chunk_id = 1

    for sample in ds:
        if written >= MAX_SAMPLES:
            break

        text = (sample.get("text") or "").strip()
        if not text:
            continue

        # strict length
        if len(text) >= MAX_CHARS:
            continue

        total_chars += len(text)

        buffer.append({
            "text": f"<|im_start|>{text}<|im_end|>"
        })
        written += 1

        # chunk flushing
        if len(buffer) >= CHUNK_SIZE:
            print(f"🧹 Writing chunk {chunk_id} ({len(buffer)}) — Total: {written}")
            write_chunk(buffer, OUTPUT_PATH)
            buffer.clear()
            chunk_id += 1

    # final chunk
    if buffer:
        print(f"🧹 Writing final chunk ({len(buffer)})")
        write_chunk(buffer, OUTPUT_PATH)

    # summary
    print("\n🎉 DONE!")
    print(f"📦 Total samples written: {written}")
    print(f"🔢 Estimated tokens: {total_chars / 4:.0f}")  # approx 4 chars/token
    print(f"🔢 Tokens (millions): {total_chars / 4 / 1e6:.3f}M")
    print(f"📄 Saved to: {OUTPUT_PATH}")


if __name__ == "__main__":
    main()

🔄 Streaming dataset: allenai/c4 (en subset)


Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

🧹 Writing chunk 1 (40000) — Total: 40000
🧹 Writing chunk 2 (40000) — Total: 80000
🧹 Writing chunk 3 (40000) — Total: 120000
🧹 Writing chunk 4 (40000) — Total: 160000
🧹 Writing chunk 5 (40000) — Total: 200000
🧹 Writing chunk 6 (40000) — Total: 240000
🧹 Writing chunk 7 (40000) — Total: 280000
🧹 Writing chunk 8 (40000) — Total: 320000
🧹 Writing chunk 9 (40000) — Total: 360000
🧹 Writing chunk 10 (40000) — Total: 400000
🧹 Writing chunk 11 (40000) — Total: 440000
🧹 Writing chunk 12 (40000) — Total: 480000
🧹 Writing final chunk (20000)

🎉 DONE!
📦 Total samples written: 500000
🔢 Estimated tokens: 39842906
🔢 Tokens (millions): 39.843M
📄 Saved to: pretrain_c4.jsonl
